In [1]:
using DataFrames # save/load txt file
Pkg.status("DataFrames")

 - DataFrames                    0.10.1


In [2]:
# make a large data and save it to data/sample100000x100.txt
include("code/makedata.jl")

In [3]:
L1 = Array{Float64,2}(readtable("data/sample5000x20.txt", header = false, separator = ' '));
L2 = Array{Float64,2}(readtable("data/sample100000x100.txt", header = false, separator = ' '));

In [4]:
include("code/mixSQP.jl");
x1 = mixSQP(L1, x = ones(size(L1,2))/size(L1,2); convtol = 1e-8,
                pqrtol = 1e-10, eps = 1e-8, sptol = 1e-3,
                maxiter = 100, maxqpiter = 100,
                lowrank = "svd", seed = 1, verbose = true)

Running SQP algorithm with the following settings:
- 5000 x 20 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-10
- partial SVD max. error = 9.68e-14
iter       objective -min(g+1) #nnz #qp
   1 -2.90875475e+03 +3.18e-01   20   0
   2 -3.21713770e+03 +6.95e+04    2  26
   3 -3.70865496e+03 +2.43e+04    2  25
   4 -3.78546754e+03 +1.04e+04    3  27
   5 -3.80605495e+03 +4.87e+03    3  19
   6 -3.82235480e+03 +2.35e+03    4  18
   7 -3.83648039e+03 +1.16e+03    4   9
   8 -3.85347774e+03 +5.77e+02    3 100
   9 -3.80679050e+03 +3.86e+04    3 100
  10 -3.81785216e+03 +1.33e+04    3  16
  11 -3.83122697e+03 +5.73e+03    3   7
  12 -3.84722766e+03 +2.67e+03    3  16
  13 -3.86039842e+03 +1.29e+03    4   6
  14 -3.87240406e+03 +6.36e+02    4   6
  15 -3.88673438e+03 +3.16e+02    5  42
  16 -3.89884057e+03 +1.58e+02    4  14
  17 -3.91060124e+03 +7.87e+01    3  23
  18 -3.92311191e+03 +3.95e+01    4  27
  19 -3.93322190e+03 +1

Dict{String,Any} with 8 entries:
  "timing"      => [0.142173, 0.00456259, 0.0023903, 0.0123554, 0.00136441, 0.0…
  "nnz"         => [20.0, 2.0, 2.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0  …  4.0, …
  "totaltime"   => 0.261833
  "lowranktime" => 0.0435189
  "obj"         => [-2908.75, -3217.14, -3708.65, -3785.47, -3806.05, -3822.35,…
  "x"           => [0.0, 0.0, 0.0, 0.0, 0.354355, 0.627714, 0.0, 0.0, 0.0, 0.0,…
  "nqp"         => [0.0, 26.0, 25.0, 27.0, 19.0, 18.0, 9.0, 100.0, 100.0, 16.0 …
  "gmin"        => [-0.317527, -69530.5, -24271.7, -10430.9, -4868.18, -2354.4,…

In [5]:
x2 = mixSQP(L2, x = ones(size(L2,2))/size(L2,2); convtol = 1e-8,
                pqrtol = 1e-10, eps = 1e-8, sptol = 1e-3,
                maxiter = 100, maxqpiter = 100,
                lowrank = "qr", seed = 1, verbose = true)

Running SQP algorithm with the following settings:
- 100000 x 100 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial QR tolerance  = 1.00e-10
- partial QR max. error = 2.69e-09
iter       objective -min(g+1) #nnz #qp
   1 5.03421516e+04 +2.90e-01  100   0
   2 5.19513613e+04 +1.06e-01    4 100
   3 4.83001092e+04 +1.57e-02    4  26
   4 4.76838683e+04 +4.20e-04    3  12
   5 4.76630722e+04 +4.85e-07    3   2
   6 4.76630469e+04 -9.29e-09    3   2
Optimization took 6 iterations and 1.4451 seconds.


Dict{String,Any} with 8 entries:
  "timing"      => [0.88527, 0.167104, 0.0322498, 0.0225479, 0.0209904, 0.0]
  "nnz"         => [100.0, 4.0, 4.0, 3.0, 3.0, 3.0]
  "totaltime"   => 1.44512
  "lowranktime" => 0.316962
  "obj"         => [50342.2, 51951.4, 48300.1, 47683.9, 47663.1, 47663.0]
  "x"           => [0.496888, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0…
  "nqp"         => [0.0, 100.0, 26.0, 12.0, 2.0, 2.0]
  "gmin"        => [-0.290037, -0.105654, -0.0156565, -0.000419683, -4.85359e-7…

In [ ]:
# You may want to check if the algorithm converges really fast for extremely large dataset
# then do
# n = 10^5; n_half = round(Int,n/2);
# srand(1); x = [randn(n_half);3*randn(n-n_half)]; s = ones(n); 
# temp = ash(x,s, mult = 1.04);
# see code/ash.jl for details 